# DeepSeek R1 的训练过程笔记
## 一、训练背景与目标
DeepSeek R1 的训练基于 R1-Zero 的经验，旨在保持强大推理能力的同时，解决纯强化学习存在的局限性（如输出可读性差、语言混合等问题）。其采用四阶段训练流程，通过结构化设计平衡推理性能与用户友好性。
## 二、四阶段训练流程
1. 第一阶段：冷启动阶段（Cold Start Stage）
核心目的：避免基础模型强化学习（RL）训练初期的不稳定，为后续 RL 提供初始策略模型。
数据收集：
收集数千个长思维链（CoT）示例，方法包括：使用少量示例提示（以长 CoT 为范例）、直接提示模型生成带反思和验证的详细答案、收集 R1-Zero 的可读格式输出并经人类注释者后处理优化。
训练方式：用上述冷启动数据对 DeepSeek-V3-Base 模型进行监督微调（SFT），得到初始 RL 策略模型（Actor）。
2. 第二阶段：强化学习聚焦推理任务
核心目的：增强模型在推理任务上的能力，同时改善语言一致性。
训练框架：基于 R1-Zero 使用的群体相对策略优化（GRPO）框架，引入额外的语言一致性奖励。
训练任务：专注于数学问题解决、编码挑战和逻辑推理，所有示例可自动生成并验证正确性（如数学结果匹配验证、代码通过单元测试验证）。
优化重点：通过语言一致性奖励，消除 R1-Zero 中常见的语言混合（如中英文切换）问题。
3. 第三阶段：扩展至通用文本生成任务
核心目的：使模型不仅在推理任务上表现优异，还能胜任通用文本生成任务（如写作、角色扮演等）。
数据来源：
推理数据：通过对第二阶段模型进行拒绝采样生成推理轨迹，经 DeepSeek-V3 质量判断后，过滤掉混合语言、长段落和代码块的思维链，保留正确响应，共约 60 万条。
非推理数据：重用 DeepSeek-V3 的部分 SFT 数据集，涵盖写作、事实问答、翻译等，约 20 万条；部分任务需生成思维链，简单查询（如 “hello”）无需。
训练方式：使用约 80 万条（60 万推理 + 20 万非推理）精选数据，对第二阶段模型进行两轮监督微调（SFT），而非继续强化学习。
4. 第四阶段：通用偏好优化强化学习
核心目的：通过强化学习使模型输出与通用偏好对齐，提升整体输出质量。
训练方式：采用通用偏好优化强化学习（如开头所述的偏好优化方法），进一步微调模型以符合人类对输出质量的期望。
## 三、训练特点总结
针对性优化：针对 R1-Zero 的局限性（可读性、语言混合），通过冷启动数据和语言一致性奖励专项改进。
数据效率：结合自动生成 / 验证数据与少量人工处理数据，平衡成本与效果。
多任务平衡：从推理任务扩展到通用文本生成，通过分阶段微调实现能力全覆盖。
框架创新：融合 GRPO 强化学习与 SFT，兼顾推理性能与训练稳定性。
